# MDA入門 2025年度 第10回演習
## 大規模言語モデル (LLM) とプロンプティング

大規模な言語モデル（LLM）の挙動を理解し，効果的に活用するための技術であるプロンプトエンジニアリングの基礎を学びます．

**演習内容:**
1. プロンプティングの実践（Zero-shot / Few-shot）
2. In-Context Learningの体験（ラベルのランダム化や順序の影響）
3. Chain-of-Thought (CoT) Promptingの実験

**使用モデル:**
本演習では，Google Colabの無償枠（T4 GPU）でも動作する軽量かつ高性能なモデル（Qwen2.5-0.5B-Instructなど）を使用します．

## 1. 環境セットアップ

演習に必要なライブラリをインストールし，環境を整えます．

In [1]:
# 必要なライブラリのインストール
!pip install -q transformers torch datasets accelerate sentencepiece

import gc
import random
import warnings

import matplotlib.pyplot as plt
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

warnings.filterwarnings("ignore")

# 日本語フォントの設定
plt.rcParams["font.sans-serif"] = ["DejaVu Sans"]
plt.rcParams["axes.unicode_minus"] = False

# シード固定（再現性のため）
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
print("セットアップ完了")

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

## 2. LLMモデルの読み込み

Colab上で十分に動作するモデルをいくつか定義し，利用可能なものを読み込みます．

In [7]:
# 試行するモデルのリスト（優先順位順）
model_candidates = [
    {
        "name": "Qwen/Qwen2.5-0.5B-Instruct",
        "params": "500M",
        "description": "Alibaba開発，多言語対応，Instructionチューニング済み",
    },
    {
        "name": "llm-jp/llm-jp-3-1.8b-instruct",
        "params": "1.8B",
        "description": "国立情報学研究所開発，日本語特化",
    },
    {
        "name": "rinna/japanese-gpt2-medium",
        "params": "330M",
        "description": "日本語GPT-2，実績豊富",
    },
    {
        "name": "cyberagent/open-calm-small",
        "params": "160M",
        "description": "サイバーエージェント開発，超軽量",
    },
]

# デバイスの確認
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用デバイス: {device}")

# モデルのロード試行
model = None
tokenizer = None
model_name = None

for i, candidate in enumerate(model_candidates, 1):
    model_name = candidate["name"]
    print(f"\n試行 {i}/4: {model_name}")
    print(f"  パラメータ数: {candidate['params']}")
    print("  読み込み中...", end="")

    try:
        # トークナイザーの読み込み
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # モデルの読み込み（メモリ効率化）
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto",
            low_cpu_mem_usage=True,
        )
        print(" [OK]")
        break

    except Exception as e:
        print(" [NG]")
        print(f"  エラー: {str(e)[:100]}...")

        # メモリをクリア
        if model is not None:
            del model
        if tokenizer is not None:
            del tokenizer
        gc.collect()
        if device == "cuda":
            torch.cuda.empty_cache()
        model = None
        tokenizer = None

if model is None:
    raise RuntimeError("すべてのモデルのロードに失敗しました．")

# テキスト生成用のパイプライン作成
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

print("\nテキスト生成パイプラインの準備完了")

使用デバイス: cuda

試行 1/4: Qwen/Qwen2.5-0.5B-Instruct
  パラメータ数: 500M
  読み込み中...

Device set to use cuda:0


 [OK]

テキスト生成パイプラインの準備完了


## 3. データセットの準備

演習で使用するサンプルデータを作成します．Amazon等の商品レビューを模したテキストと，その感情ラベル（positive/negative/neutral）のペアを用意します．

In [8]:
dataset = [
    {"text": "この商品は最高です！買って良かったです．", "label": "positive"},
    {"text": "期待外れでした．品質が悪すぎます．", "label": "negative"},
    {"text": "値段の割には普通です．可もなく不可もなく．", "label": "neutral"},
    {"text": "素晴らしい製品です．友人にも勧めたいと思います．", "label": "positive"},
    {"text": "サービスが最悪でした．二度と利用しません．", "label": "negative"},
    {"text": "機能は良いですが，デザインがイマイチです．", "label": "neutral"},
    {"text": "期待以上の性能でした．大満足です！", "label": "positive"},
    {"text": "説明と実物が全然違う．詐欺レベルです．", "label": "negative"},
    {"text": "この価格なら妥当だと思います．", "label": "neutral"},
    {"text": "感動しました．本当に買って良かったです．", "label": "positive"},
]

print(f"サンプルデータ作成完了（{len(dataset)}件）")
print("【データ例】")
print(dataset[0])

サンプルデータ作成完了（10件）
【データ例】
{'text': 'この商品は最高です！買って良かったです．', 'label': 'positive'}


## 4. 演習1: プロンプティングの基礎実践 (Zero-shot)

**Zero-shot prompting**とは，LLMに対して例題を与えずに，直接タスクの指示のみを行って回答を生成させる手法です．
まずはこちらを試してみましょう．

In [9]:
# テスト用のテキストを選択
test_text = dataset[0]["text"]
test_label = dataset[0]["label"]

print(f"テキスト: {test_text}")
print(f"正解ラベル: {test_label}")
print("-" * 50)

# Zero-shot プロンプトの作成
zero_shot_prompt = f"""以下のテキストの感情を「ポジティブ」「ネガティブ」「中立」のいずれかで分類してください．

テキスト: {test_text}

感情:"""

print("プロンプト：")
print(zero_shot_prompt)
print("-" * 50)

# テキスト生成
output = generator(
    zero_shot_prompt,
    max_new_tokens=20,
    num_return_sequences=1,
    temperature=0.3,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
    truncation=True,
)

generated_text = output[0]["generated_text"]
# 入力プロンプトを除いた生成部分のみを抽出
result = generated_text[len(zero_shot_prompt):].strip()

print(f"モデルの回答: {result}")

テキスト: この商品は最高です！買って良かったです．
正解ラベル: positive
--------------------------------------------------
プロンプト：
以下のテキストの感情を「ポジティブ」「ネガティブ」「中立」のいずれかで分類してください．

テキスト: この商品は最高です！買って良かったです．

感情:
--------------------------------------------------
モデルの回答: 感情: 感情: 中立

理由：このテキストは


### 【考察】
実行結果を見て，Zero-shotプロンプティングの特徴や課題について考えてみましょう．正しく分類できましたか？また，余計な出力が含まれていたりしませんか？

（ここに考察を記入してください）


## 5. 演習2: In-Context Learning (ICL) の実験

**Few-shot prompting**（In-Context Learning）では，プロンプトの中にいくつかの例題（Example）を含めることで，モデルにタスクの形式やパターンを学習させます．

ここでは以下の3つの条件で実験を行い，結果を比較します：
1. **標準的なFew-shot**: 正解ラベルを使用した通常のFew-shot
2. **ラベルランダム化**: 例題のラベルをデタラメに入れ替えたもの
3. **順序変更**: 例題の提示順序をシャッフルしたもの

これにより，モデルが「ラベルの意味」を理解しているのか，それとも「形式」を重視しているのかといった性質を探ります．

In [16]:
# ICL用のヘルパー関数定義

def generate_prediction(prompt, generator, tokenizer):
    try:
        output = generator(
            prompt,
            max_new_tokens=5, # 短い回答のみ取得
            num_return_sequences=1,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id if tokenizer.eos_token_id else tokenizer.pad_token_id,
            truncation=True
        )
        generated = output[0]["generated_text"][len(prompt):].strip()
        return generated.split('\n')[0].strip()
    except:
        return "Error"

def build_prompt(examples, target_text, randomize_labels=False):
    prompt = "以下のテキストの感情を「ポジティブ」「ネガティブ」「中立」で分類してください．\n\n"
    labels = ["positive", "negative", "neutral"]

    for i, ex in enumerate(examples):
        txt = ex["text"]
        lbl = ex["label"]

        if randomize_labels:
            lbl = random.choice(labels)

        prompt += f"例{i+1}:\nテキスト: {txt}\n感情: {lbl}\n\n"

    prompt += f"テキスト: {target_text}\n感情:"
    return prompt

# --------------------
# 実験実行
# --------------------

n_shots = 5
icl_examples = dataset[:n_shots]      # 例示用データ
icl_test_data = dataset[n_shots:n_shots+5] # テスト用データ

conditions = [
    {"name": "標準Few-shot", "random_label": False, "shuffle_order": False},
    {"name": "ラベルランダム", "random_label": True, "shuffle_order": False},
    {"name": "順序変更",     "random_label": False, "shuffle_order": True},
]

results = {c["name"]: [] for c in conditions}

print(f"評価データ数: {len(icl_test_data)}件")
print("-" * 60)

for idx, sample in enumerate(icl_test_data):
    test_text = sample["text"]
    true_label = sample["label"]

    print(f"\n◆テストケース {idx+1}: {test_text[:30]}... (正解: {true_label})")

    for cond in conditions:
        current_examples = list(icl_examples)

        if cond["shuffle_order"]:
            random.shuffle(current_examples)

        prompt = build_prompt(
            current_examples,
            test_text,
            randomize_labels=cond["random_label"]
        )

        pred = generate_prediction(prompt, generator, tokenizer)
        is_correct = (true_label in pred)
        results[cond["name"]].append(is_correct)

        mark = "[OK]" if is_correct else "[NG]"
        print(f"    - {cond['name']}: {pred} {mark}")

print("\n" + "=" * 60)
print("【実験結果サマリ（正解率）】")
for name, res in results.items():
    if len(res) > 0:
        acc = sum(res) / len(res) * 100
        print(f"{name}: {acc:.1f}%")
    else:
        print(f"{name}: データなし")

評価データ数: 5件
------------------------------------------------------------

◆テストケース 1: 機能は良いですが，デザインがイマイチです．... (正解: neutral)
    - 標準Few-shot: negative [NG]
    - ラベルランダム: neutral [OK]
    - 順序変更: negative [NG]

◆テストケース 2: 期待以上の性能でした．大満足です！... (正解: positive)
    - 標準Few-shot: positive [OK]
    - ラベルランダム: positive [OK]
    - 順序変更: positive [OK]

◆テストケース 3: 説明と実物が全然違う．詐欺レベルです．... (正解: negative)
    - 標準Few-shot: negative [OK]
    - ラベルランダム: negative [OK]
    - 順序変更: negative [OK]

◆テストケース 4: この価格なら妥当だと思います．... (正解: neutral)
    - 標準Few-shot: neutral [OK]
    - ラベルランダム: neutral [OK]
    - 順序変更: neutral [OK]

◆テストケース 5: 感動しました．本当に買って良かったです．... (正解: positive)
    - 標準Few-shot: positive [OK]
    - ラベルランダム: positive [OK]
    - 順序変更: positive [OK]

【実験結果サマリ（正解率）】
標準Few-shot: 80.0%
ラベルランダム: 100.0%
順序変更: 80.0%


### 【考察】
1. Few-shotとZero-shotの結果を比較して，どのような変化がありましたか．
2. ラベルをランダムに入れ替えた場合と正しいラベルの場合で，結果にどのような違い（あるいは共通点）が見られましたか．そこからICLの性質について何が言えますか．

（ここに考察を記入してください）


## 6. 演習3: Chain-of-Thought (CoT) Prompting

**Chain-of-Thought (CoT)** は，モデルに「段階的に考える」ことを促す手法です．推論過程（Reasoning）を明示的に出力させることで，複雑な問題（算数や論理問題など）の正答率を向上させることができます．

ここでは算数の文章題を用いて，以下の3パターンを比較します：
1. **通常Few-shot**: 答えのみを例示
2. **CoT Few-shot**: 例示の中に「考え方」を含める
3. **Zero-shot CoT**: 例示なしで「段階的に考えましょう」とだけ指示する

In [17]:
# 問題定義
test_problem = {
    "question": "本屋で500円の本を2冊と300円の雑誌を1冊買いました．合計金額はいくらですか？",
    "answer": "1300円",
}

math_problems = [
    {
        "question": "太郎は5個のりんごを持っていました．3個もらいました．今何個ありますか？",
        "answer": "8個",
        "reasoning": "最初に5個持っていて，3個もらったので，5 + 3 = 8個です．",
    },
    {
        "question": "花子は10個のみかんを持っていました．4個食べました．今何個ありますか？",
        "answer": "6個",
        "reasoning": "最初に10個持っていて，4個食べたので，10 - 4 = 6個です．",
    },
]

print(f"問題: {test_problem['question']}")
print(f"正解: {test_problem['answer']}")
print("-" * 50)

# 1. 通常Few-shot
standard_prompt = "以下の算数の問題を解いてください．\n\n"
for ex in math_problems:
    standard_prompt += f"問題: {ex['question']}\n答え: {ex['answer']}\n\n"
standard_prompt += f"問題: {test_problem['question']}\n答え:"

# 2. CoT Few-shot
cot_prompt = "以下の算数の問題を，計算過程を示しながら解いてください．\n\n"
for ex in math_problems:
    cot_prompt += f"問題: {ex['question']}\n考え方: {ex['reasoning']}\n答え: {ex['answer']}\n\n"
cot_prompt += f"問題: {test_problem['question']}\n考え方:"

# 3. Zero-shot CoT
zero_shot_cot_prompt = f"""以下の算数の問題を解いてください．段階的に考えましょう．

問題: {test_problem['question']}

考え方:"""

# 各プロンプトでの実行
prompts = {
    "通常Few-shot": standard_prompt,
    "CoT Few-shot": cot_prompt,
    "Zero-shot CoT": zero_shot_cot_prompt
}

for name, prompt in prompts.items():
    print(f"\n【{name}】")
    try:
        out = generator(
            prompt,
            max_new_tokens=60,
            num_return_sequences=1,
            temperature=0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            truncation=True,
        )
        res = out[0]["generated_text"][len(prompt):].strip()
        print(f"出力: {res[:100]}...")
    except:
        print("エラーが発生しました")

問題: 本屋で500円の本を2冊と300円の雑誌を1冊買いました．合計金額はいくらですか？
正解: 1300円
--------------------------------------------------

【通常Few-shot】
出力: 700円です。

問題: 食べ物店で100円のチーズを1本买了。また、そのチーズを10本分切って、100円の小鉢を作りました。食料品の価格はどれ...

【CoT Few-shot】
出力: まず、500円の本を2冊买了なので，500 × 2 = 1000円でした。さらに、1冊の雑誌が300円でしたので，1000 + 300 = 13...

【Zero-shot CoT】
出力: まず、本屋で購入した物の価格を計算します。500円の本を2冊买了り、300円の雑誌を1冊买了りすると、それぞれの価格が同じです。

本の価格 = 5...


### 【考察】
推論過程（考え方）をプロンプトに含めることで，モデルの出力はどう変化しましたか．単に答えが合うかどうかだけでなく，その過程が論理的かどうかも確認してください．

（ここに考察を記入してください）


## 演習まとめ

本演習を通じて，以下のことが確認できました．

1. **プロンプト設計の重要性**: 指示の仕方一つでモデルの出力精度が大きく変わること．
2. **ICLの特性**: LLMは例題から「タスクの形式」を強く学習し，ラベルの意味よりも形式的な整合性を重視する場合があること（Label Chaos）．
3. **CoTの効果**: 計算などの推論タスクでは，思考過程を言語化させること（CoT）で性能が向上すること．

### 参考文献
- [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)
- [Thinking about prompting (Min et al., 2022)](https://arxiv.org/abs/2202.12837)
- [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models (Wei et al., 2022)](https://arxiv.org/abs/2201.11903)